In [65]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

In [66]:
file_name = '네트워크4(200128,v8,경유정류장순서반영,김민기).xlsx'
df_link_id = pd.read_excel(file_name, sheet_name='Link_ID', dtype={'ROUTE_ID': str})
df_visum_link = pd.read_excel(file_name, sheet_name='Visum_link')
df_visum_stop = pd.read_excel(file_name, sheet_name='Visum_stop')

In [3]:
df_columns = ['$LINEROUTEITEM:LINENAME', 'LINEROUTENAME', 'DIRECTIONCODE', 'INDEX', 'ISROUTEPOINT', 'NODENO', 'STOPPOINTNO', 'POSTLENGTH', 'ADDVAL']
df_line_route_items = pd.DataFrame(columns=df_columns)
df_line_route_items.to_csv('line_route_items.csv', index=False, encoding='utf-8-sig')

DIRECTIONCODE = '>'
POSTLENGTH = '0km'
ADDVAL = 0

INDEX = 1; N_CNT = 0; S_CNT = 0
F_NODE = np.nan; T_NODE = np.nan; prev_route_id = np.nan; prev_stop = np.nan; prev_link = np.nan; prev_link_no = np.nan;

df_link = df_link_id
# df_link = df_link_id[df_link_id['ROUTE_NAME'] == '1100']
# df_link = df_link_id[(df_link_id['ROUTE_NAME'] == '1') | (df_link_id['ROUTE_NAME'] == '1301') | (df_link_id['ROUTE_NAME'] == '1100') | (df_link_id['ROUTE_NAME'] == '320')]
for _, l in tqdm_notebook(df_link.iterrows()):
    route_id = l['ROUTE_ID']; route_name = l['ROUTE_NAME']; link_id = l['LINK_ID']; link_idx = l['INDEX']

    if prev_route_id != route_id:
        for i, item in df_line_route_items.iloc[::-1].iterrows():
            if item['ISROUTEPOINT'] == 1:
                break;
            df_line_route_items = df_line_route_items.drop(i, 0)
        df_line_route_items.to_csv('line_route_items.csv', mode='a', index=False, encoding='utf-8-sig', header=False)
        df_line_route_items = pd.DataFrame(columns=df_columns) 
        INDEX = 1; N_CNT = 0; S_CNT = 0
        F_NODE = np.nan; T_NODE = np.nan; prev_stop = np.nan; prev_link = np.nan; prev_link_no = np.nan;
    prev_route_id = route_id
    
    if prev_link == link_id:
        continue
    prev_link = link_id
    
    n = len(set(df_visum_link[(df_visum_link['LINK_ID'] == link_id)]['LINK_NO']))    
    for i in range(n):
        if np.isnan(T_NODE):
            df_vlink = df_visum_link[df_visum_link['LINK_ID'] == link_id]
        else:
            df_vlink = df_visum_link[(df_visum_link['LINK_ID'] == link_id) & (df_visum_link['F_NODE'] == T_NODE) & (df_visum_link['T_NODE'] != F_NODE) ]
        
        for _, vl in df_vlink.iterrows():
            link_no = vl['LINK_NO']
            if prev_link_no == link_no:
                continue
            prev_link_no = link_no
            
            if np.isnan(T_NODE):
                df_vstop = (df_visum_stop[df_visum_stop['LINK_NO'] == link_no]).sort_values(by=['INDEX', 'F_NODE'])
            else:
                df_vstop = (df_visum_stop[(df_visum_stop['LINK_NO'] == link_no) & (df_visum_stop['F_NODE'] == T_NODE)]).sort_values(by=['INDEX', 'F_NODE'])
            
            for _, vs in df_vstop.iterrows():
                if prev_stop == vs['STOP_NO']:
                    continue
                prev_stop = vs['STOP_NO']
                
                if INDEX == 1:
                    F_NODE = vs['F_NODE']
                    T_NODE = df_visum_link[(df_visum_link['LINK_ID'] == link_id) & (df_visum_link['F_NODE'] == F_NODE) & (df_visum_link['LINK_NO'] == link_no)].T_NODE.item()                
                df_line_route_items = df_line_route_items.append(pd.Series([route_id, route_name, DIRECTIONCODE, INDEX, 1, '', vs['STOP_NO'], POSTLENGTH, ADDVAL], index=df_columns), ignore_index=True)
#                 print(vs['STOP_NO'])
                INDEX = INDEX + 1 
                S_CNT = S_CNT + 1
            
            if S_CNT > 0 and N_CNT == 0:
                if F_NODE != T_NODE:
                    df_line_route_items = df_line_route_items.append(pd.Series([route_id, route_name, DIRECTIONCODE, INDEX, 0, T_NODE, '', POSTLENGTH, ADDVAL], index=df_columns), ignore_index=True)
                    INDEX = INDEX + 1
                    N_CNT = N_CNT + 1
                    prev_stop = np.nan
                break
            else:
                F_NODE = T_NODE
                T_NODE = vl['T_NODE']
                if F_NODE != T_NODE and INDEX > 1:
                    df_line_route_items = df_line_route_items.append(pd.Series([route_id, route_name, DIRECTIONCODE, INDEX, 0, T_NODE, '', POSTLENGTH, ADDVAL], index=df_columns), ignore_index=True)
                    INDEX = INDEX + 1
                    N_CNT = N_CNT + 1
                    prev_stop = np.nan
            
#             print(F_NODE, '->', T_NODE)
                
for i, item in df_line_route_items.iloc[::-1].iterrows():
    if item['ISROUTEPOINT'] == 1:
        break;
    df_line_route_items = df_line_route_items.drop(i, 0)
df_line_route_items.to_csv('line_route_items.csv', mode='a', index=False, encoding='utf-8-sig', header=False)

In [67]:
df_time_item = pd.read_excel(file_name, sheet_name='Time', dtype={'노선ID': str, 'ISC노선ID': str, '정류소순번': str, '정류소ID': str, 'VISUM정류소NO': str})

In [68]:
df_columns = ['$TIMEPROFILEITEM:LINENAME', 'LINEROUTENAME', 'DIRECTIONCODE', 'TIMEPROFILENAME', 'INDEX', 'LRITEMINDEX', 'ALIGHT', 'BOARD', 'ARR', 'DEP', 'NUMFAREPOINTS', 'NUMFAREPOINTSBOARD', 'NUMFAREPOINTSTHROUGH', 'NUMFAREPOINTSALIGHT', 'ADDVAL']
df_time_profile_items = pd.DataFrame(columns=df_columns)
df_time_profile_items.to_csv('time_profile_items.csv', index=False, encoding='utf-8-sig')

ARR = '00:00:00'
DEP = '00:00:00'

INDEX = 1; N_CNT = 0; S_CNT = 0
F_NODE = np.nan; T_NODE = np.nan; prev_route_id = np.nan; prev_stop = np.nan; prev_link = np.nan; prev_link_no = np.nan;

df_link = df_link_id
# df_link = df_link_id[(df_link_id['ROUTE_NAME'] == '1') | (df_link_id['ROUTE_NAME'] == '1301') | (df_link_id['ROUTE_NAME'] == '1100') | (df_link_id['ROUTE_NAME'] == '320')]
for _, l in tqdm_notebook(df_link.iterrows()):
    route_id = l['ROUTE_ID']; route_name = l['ROUTE_NAME']; link_id = l['LINK_ID']; link_idx = l['INDEX']

    if prev_route_id != route_id:
        ARR = '00:00:00'
        DEP = '00:00:00'
        df_time_profile_items.to_csv('time_profile_items.csv', mode='a', index=False, encoding='utf-8-sig', header=False)
        df_time_profile_items = pd.DataFrame(columns=df_columns)
        INDEX = 1; N_CNT = 0; S_CNT = 0
        F_NODE = np.nan; T_NODE = np.nan; prev_stop = np.nan; prev_link = np.nan; prev_link_no = np.nan;
        df_time = df_time_item[df_time_item['ISC노선ID'] == route_id]
    prev_route_id = route_id
    
    if prev_link == link_id:
        continue
    prev_link = link_id
    
    n = len(set(df_visum_link[(df_visum_link['LINK_ID'] == link_id)]['LINK_NO']))    
    for i in range(n):
        if np.isnan(T_NODE):
            df_vlink = df_visum_link[df_visum_link['LINK_ID'] == link_id]
        else:
            df_vlink = df_visum_link[(df_visum_link['LINK_ID'] == link_id) & (df_visum_link['F_NODE'] == T_NODE) & (df_visum_link['T_NODE'] != F_NODE) ]
        
        for _, vl in df_vlink.iterrows():
            link_no = vl['LINK_NO']
            if prev_link_no == link_no:
                continue
            prev_link_no = link_no
            
            if np.isnan(T_NODE):
                df_vstop = (df_visum_stop[df_visum_stop['LINK_NO'] == link_no]).sort_values(by=['INDEX', 'F_NODE'])
            else:
                df_vstop = (df_visum_stop[(df_visum_stop['LINK_NO'] == link_no) & (df_visum_stop['F_NODE'] == T_NODE)]).sort_values(by=['INDEX', 'F_NODE'])
            
            for _, vs in df_vstop.iterrows():
                if prev_stop == vs['STOP_NO']:
                    continue
                prev_stop = vs['STOP_NO']

                if INDEX == 1:
                    F_NODE = vs['F_NODE']
                    T_NODE = df_visum_link[(df_visum_link['LINK_ID'] == link_id) & (df_visum_link['F_NODE'] == F_NODE) & (df_visum_link['LINK_NO'] == link_no)].T_NODE.item()                
                stop_info = df_time[df_time['VISUM정류소NO'] == str(vs['STOP_NO'])].head(1)
                if len(stop_info) != 0:
                    df_time_profile_items = df_time_profile_items.append(pd.Series([route_id, route_name, '>', 1, INDEX, vs['STOP_NO'], 1, 1, stop_info['ARR'].item(), stop_info['DEP'].item(), 0, 0, 0, 0, 0], index=df_columns), ignore_index=True)
                    ARR = stop_info['DEP'].item()
                    DEP = ARR
                    df_time = df_time.drop(stop_info.index)
                else:
                    df_time_profile_items = df_time_profile_items.append(pd.Series([route_id, route_name, '>', 1, INDEX, vs['STOP_NO'], 0, 0, ARR, DEP, 0, 0, 0, 0, 0], index=df_columns), ignore_index=True)                

                INDEX = INDEX + 1 
                S_CNT = S_CNT + 1
            
            if S_CNT > 0 and N_CNT == 0:
                if F_NODE != T_NODE:
#                     df_line_route_items = df_line_route_items.append(pd.Series([route_id, route_name, DIRECTIONCODE, INDEX, 0, T_NODE, '', POSTLENGTH, ADDVAL], index=df_columns), ignore_index=True)
#                     INDEX = INDEX + 1
                    N_CNT = N_CNT + 1
                    prev_stop = np.nan
                break
            else:
                F_NODE = T_NODE
                T_NODE = vl['T_NODE']
                if F_NODE != T_NODE and INDEX > 1:
#                     df_line_route_items = df_line_route_items.append(pd.Series([route_id, route_name, DIRECTIONCODE, INDEX, 0, T_NODE, '', POSTLENGTH, ADDVAL], index=df_columns), ignore_index=True)
#                     INDEX = INDEX + 1
                    N_CNT = N_CNT + 1
                    prev_stop = np.nan
                
df_time_profile_items.to_csv('time_profile_items.csv', mode='a', index=False, encoding='utf-8-sig', header=False)